In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Drug;
DROP TABLE IF EXISTS Pathology;
DROP TABLE IF EXISTS DrugUse;

CREATE TABLE Drug (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../data/faers-2017/drug.csv');

CREATE TABLE Pathology (
  code INT NOT NULL,
  name VARCHAR(200) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../data/faers-2017/pathology.csv');

CREATE TABLE DrugUse (
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  codeDrug INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(codeDrug)
    REFERENCES Drug(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
) AS SELECT
    idPerson,
    codePathology,
    codeDrug
FROM CSVREAD('../../data/faers-2017/drug-use.csv');

CREATE TABLE SideEffect(
  idPerson INT NOT NULL,
  codePathology INT NOT NULL,
  FOREIGN KEY(codePathology)
    REFERENCES Pathology(code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT idPerson, codePathology
FROM CSVREAD('../../data/faers-2017/sideeffect.csv');

In [3]:
SELECT * FROM Drug;
SELECT * FROM Pathology;
SELECT * FROM DrugUse;
SELECT * FROM SideEffect;

# FDA Adverse Event Reporting System (FAERS)

Considere a base de dados que reúne relatos de efeitos adversos de drogas: [FDA Adverse Event Reporting System (FAERS)](https://open.fda.gov/data/faers/).

As fontes de dados são publicadas e documentadas em: [Latest Quarterly Data Files](https://www.fda.gov/drugs/fda-adverse-event-reporting-system-faers/fda-adverse-event-reporting-system-faers-latest-quarterly-data-files)

# Interação entre medicamentos
Considere como uma interação toda vez que um medicamento A é tomado em conjunto com um medicamento B por uma pessoa. Crie uma consulta que retorne para cada par de medicamento A,B o número de interações, somente para aqueles medicamentos que são tomados em conjunto mais de 30 vezes (passaremos a chamar de **grupo mais 30**).

In [4]:
DROP VIEW IF EXISTS PersonDrugUse;
CREATE VIEW PersonDrugUse AS
SELECT idPerson, codeDrug
    FROM DrugUse
    GROUP BY idPerson,codeDrug;
SELECT * FROM PersonDrugUse;

In [5]:
-- Tempo de execução muito longo, pode estourar a memória no binder
-- DROP VIEW IF EXISTS DrugInteraction;
-- CREATE VIEW DrugInteraction AS
-- SELECT PDU1.codedrug druga, PDU2.codedrug drugb, COUNT(*) weight
--     FROM PersonDrugUse as PDU1, PersonDrugUse as PDU2
--     WHERE PDU1.idPerson = PDU2.idPerson AND
--           PDU1.codeDrug < PDU2.codeDrug
--     GROUP BY PDU1.codedrug, PDU2.codedrug;
-- SELECT * FROM DrugI30;

In [6]:
DROP TABLE IF EXISTS DrugInteraction;
CREATE TABLE DrugInteraction (
  druga INT NOT NULL,
  drugb INT NOT NULL,
  weight INT NOT NULL,
  PRIMARY KEY(druga, drugb)
) AS SELECT
    druga,
    drugb,
    weight
FROM CSVREAD('../../data/faers-2017/results/my-drug-interaction.csv');

In [7]:
CREATE VIEW IF NOT EXISTS DrugI30 AS
SELECT * FROM DrugInteraction WHERE weight > 30;

## Tabela Resultante

Se estiver demorando muito para gerar a tabela, você poderá usar a seguinte tabela resultante pronta. Essa tabela tem todas as interações, você ainda precisará filtrar o **grupo mais 30**.

In [8]:
-- NÃO ESTÁ SENDO UTILIZADA
-- CREATE TABLE DrugInteraction (
--   druga INT NOT NULL,
--   drugb INT NOT NULL,
--   weight INT NOT NULL,
--   PRIMARY KEY(druga, drugb)
-- ) AS SELECT
--     druga,
--     drugb,
--     weight
-- FROM CSVREAD('../../data/faers-2017/results/drug-interaction.csv');

-- SELECT * FROM DrugInteraction;

# Suporte

### Qual o suporte que cada uma das associações tem?

Gere uma consulta que apresente o suporte do **grupo mais 30**. Ordene o resultado por ordem de suporte. 

Observação: Para que o resultado da divisão seja fracionária, converta o numerador em `DOUBLE` usando:
~~~sql
CAST(campo AS DOUBLE)
~~~

In [9]:
SELECT druga, drugb, (CAST(weight AS DOUBLE)/(SELECT COUNT(*) FROM (SELECT COUNT(*) FROM PersonDrugUse GROUP BY idPerson))) suporte
    FROM DrugInteraction
    WHERE weight > 30
    ORDER BY suporte DESC;

In [10]:
SELECT COUNT(*) FROM (SELECT COUNT(*) FROM PersonDrugUse GROUP BY idPerson);
SELECT DISTINCT COUNT(idPerson) FROM DrugUse

# Confiança

### Qual a confiança que cada uma das associações tem?

Gere uma consulta que apresente a confiança do **grupo mais 30**. Ordene o resultado por ordem de confiança.

In [11]:
SELECT DI.druga A, DI.drugb B,(CAST(DI.weight AS DOUBLE)/(COUNT(PDU.codedrug))) confianca
    FROM DrugInteraction DI, PersonDrugUse PDU
    WHERE DI.weight > 30 AND
          PDU.codedrug = DI.druga
    GROUP BY DI.druga, DI.drugb
UNION
SELECT DI.drugb A, DI.druga B,(CAST(DI.weight AS DOUBLE)/(COUNT(PDU.codedrug))) confianca
    FROM DrugInteraction DI, PersonDrugUse PDU
    WHERE DI.weight > 30 AND
          PDU.codedrug = DI.drugb
    GROUP BY DI.druga, DI.drugb
    ORDER BY confianca DESC;

# Efeito Colateral

Para cada medicamento, qual o percentual do medicamento A que tem efeito colateral junto com o medicamento B, quando comparado com seu uso sem o medicamento B.

In [12]:
CREATE VIEW IF NOT EXISTS PersonSideEffects AS
SELECT idPerson, COUNT(*) sideEffects
    FROM SideEffect
    GROUP BY idPerson;
SELECT * FROM PersonSideEffects;

In [13]:
SELECT DI.druga A, DI.drugb B, SUM(PSE.sideEffects)/
    ((SELECT SUM (PSE2.sideEffects) FROM PersonSideEffects PSE2, PersonDrugUse PDU3 WHERE PDU3.codedrug = DI.druga AND PSE2.idPerson = PDU3.idPerson) 
    -SUM(PSE.sideEffects)) AB_woB
    FROM DrugInteraction DI, PersonDrugUse PDU1, PersonDrugUse PDU2, PersonSideEffects PSE
    WHERE DI.weight > 30 AND
          PDU1.codedrug = DI.druga AND
          PDU2.codedrug = DI.drugb AND
          PDU1.idPerson = PDU2.idPerson AND
          PSE.idPerson = PDU1.idPerson 
    GROUP BY DI.druga, DI.drugb
    
UNION

SELECT DI.drugb A, DI.druga B, CAST(SUM(PSE.sideEffects) AS DOUBLE)/
    ((SELECT SUM (PSE2.sideEffects) FROM PersonSideEffects PSE2, PersonDrugUse PDU3 WHERE PDU3.codedrug = DI.drugb AND PSE2.idPerson = PDU3.idPerson) 
    -SUM(PSE.sideEffects)) AB_woB
    FROM DrugInteraction DI, PersonDrugUse PDU1, PersonDrugUse PDU2, PersonSideEffects PSE
    WHERE DI.weight > 30 AND
          PDU1.codedrug = DI.druga AND
          PDU2.codedrug = DI.drugb AND
          PDU1.idPerson = PDU2.idPerson AND
          PSE.idPerson = PDU1.idPerson 
    GROUP BY DI.druga, DI.drugb
    ORDER BY AB_woB DESC;